In [1]:
!pip install tensorflow opencv-contrib-python youtube-dl moviepy pydot
!pip install git+https://github.com/tensorflow/docs
!pip install pandas
!pip install imutils

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-sx7pu_m6
  Running command git clone --filter=blob:none -q https://github.com/tensorflow/docs /tmp/pip-req-build-sx7pu_m6
  Resolved https://github.com/tensorflow/docs to commit d8382c97b29152391a647713e1c35d10dae03552
  Preparing metadata (setup.py) ... done


In [2]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from imutils import paths
import tensorflow as tf
import imageio
import cv2


dataset_path = os.listdir('/home/isfrc/Desktop/cap/model-4/train1')

label_types = os.listdir('/home/isfrc/Desktop/cap/model-4/train1')
print (label_types)  

['walking', 'closing_opening_door', 'handshaking', 'typing', 'sleeping']


In [3]:
rooms = []

for item in dataset_path:
 
 all_rooms = os.listdir('/home/isfrc/Desktop/cap/model-4/train1' + '/' +item)

 
 for room in all_rooms:
    rooms.append((item, str('/home/isfrc/Desktop/cap/model-4/train1' + '/' +item) + '/' + room))
    
        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

       tag                                         video_name
0  walking  /home/isfrc/Desktop/cap/model-4/train1/walking...
1  walking  /home/isfrc/Desktop/cap/model-4/train1/walking...
2  walking  /home/isfrc/Desktop/cap/model-4/train1/walking...
3  walking  /home/isfrc/Desktop/cap/model-4/train1/walking...
4  walking  /home/isfrc/Desktop/cap/model-4/train1/walking...
          tag                                         video_name
607  sleeping  /home/isfrc/Desktop/cap/model-4/train1/sleepin...
608  sleeping  /home/isfrc/Desktop/cap/model-4/train1/sleepin...
609  sleeping  /home/isfrc/Desktop/cap/model-4/train1/sleepin...
610  sleeping  /home/isfrc/Desktop/cap/model-4/train1/sleepin...
611  sleeping  /home/isfrc/Desktop/cap/model-4/train1/sleepin...


In [4]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('/home/isfrc/Desktop/cap/model-4/train1.csv')

In [7]:
train_df = pd.read_csv("/home/isfrc/Desktop/cap/model-4/train1.csv")
test_df = pd.read_csv("/home/isfrc/Desktop/cap/model-4/test1.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 612
Total videos for testing: 75


,Unnamed: 0,video_name,tag
244,244,/home/isfrc/Desktop/cap/model-4/train1/handsha...,handshaking
444,444,/home/isfrc/Desktop/cap/model-4/train1/typing/...,typing
293,293,/home/isfrc/Desktop/cap/model-4/train1/handsha...,handshaking
397,397,/home/isfrc/Desktop/cap/model-4/train1/typing/...,typing
272,272,/home/isfrc/Desktop/cap/model-4/train1/handsha...,handshaking
478,478,/home/isfrc/Desktop/cap/model-4/train1/typing/...,typing
71,71,/home/isfrc/Desktop/cap/model-4/train1/walking...,walking
487,487,/home/isfrc/Desktop/cap/model-4/train1/typing/...,typing
311,311,/home/isfrc/Desktop/cap/model-4/train1/handsha...,handshaking
159,159,/home/isfrc/Desktop/cap/model-4/train1/closing...,closing_opening_door


In [8]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [9]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [10]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['closing_opening_door', 'handshaking', 'sleeping', 'typing', 'walking']


array([[4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
       [4],
    

In [11]:

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 50

MAX_SEQ_LENGTH = 60
NUM_FEATURES = 2048

In [13]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    labels = df["tag"].values

    labels = label_processor(labels[..., None]).numpy()

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")


    for idx, path in enumerate(video_paths):
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train1")
test_data, test_labels = prepare_all_videos(test_df, "test1")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")


Frame features in train set: (612, 60, 2048)
Frame masks in train set: (612, 60)
train_labels in train set: (612, 1)
test_labels in train set: (75, 1)


In [16]:

def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 50
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()




Epoch 1/100
18/18 [==============================] - 6s 132ms/step - loss: 1.4812 - accuracy: 0.4109 - val_loss: 2.4980 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 2.49798, saving model to ./tmp/video_classifier
Epoch 2/100
18/18 [==============================] - 1s 48ms/step - loss: 1.3353 - accuracy: 0.5109 - val_loss: 2.4382 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 2.49798 to 2.43824, saving model to ./tmp/video_classifier
Epoch 3/100
18/18 [==============================] - 1s 47ms/step - loss: 1.2313 - accuracy: 0.5564 - val_loss: 2.1881 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss improved from 2.43824 to 2.18808, saving model to ./tmp/video_classifier
Epoch 4/100
18/18 [==============================] - 1s 48ms/step - loss: 1.1549 - accuracy: 0.5945 - val_loss: 2.0902 - val_accuracy: 0.0000e+00

Epoch 00004: val_loss improved from 2.18808 to 2.09018, saving model to ./tmp/video_classifier
Epoch 5/100
18/18 [================


Epoch 00039: val_loss did not improve from 1.02979
Epoch 40/100
18/18 [==============================] - 1s 52ms/step - loss: 0.1861 - accuracy: 0.9455 - val_loss: 1.8488 - val_accuracy: 0.3548

Epoch 00040: val_loss did not improve from 1.02979
Epoch 41/100
18/18 [==============================] - 1s 51ms/step - loss: 0.2514 - accuracy: 0.9236 - val_loss: 2.3832 - val_accuracy: 0.2258

Epoch 00041: val_loss did not improve from 1.02979
Epoch 42/100
18/18 [==============================] - 1s 51ms/step - loss: 0.2003 - accuracy: 0.9255 - val_loss: 1.6309 - val_accuracy: 0.5323

Epoch 00042: val_loss did not improve from 1.02979
Epoch 43/100
18/18 [==============================] - 1s 56ms/step - loss: 0.2620 - accuracy: 0.9200 - val_loss: 1.0706 - val_accuracy: 0.5323

Epoch 00043: val_loss did not improve from 1.02979
Epoch 44/100
18/18 [==============================] - 1s 51ms/step - loss: 0.1542 - accuracy: 0.9509 - val_loss: 1.5868 - val_accuracy: 0.5000

Epoch 00044: val_loss di


Epoch 00080: val_loss did not improve from 0.57795
Epoch 81/100
18/18 [==============================] - 1s 52ms/step - loss: 0.0810 - accuracy: 0.9655 - val_loss: 1.7848 - val_accuracy: 0.6452

Epoch 00081: val_loss did not improve from 0.57795
Epoch 82/100
18/18 [==============================] - 1s 51ms/step - loss: 0.0571 - accuracy: 0.9782 - val_loss: 1.6407 - val_accuracy: 0.6452

Epoch 00082: val_loss did not improve from 0.57795
Epoch 83/100
18/18 [==============================] - 1s 53ms/step - loss: 0.0540 - accuracy: 0.9818 - val_loss: 1.8268 - val_accuracy: 0.6452

Epoch 00083: val_loss did not improve from 0.57795
Epoch 84/100
18/18 [==============================] - 1s 51ms/step - loss: 0.0611 - accuracy: 0.9764 - val_loss: 1.6758 - val_accuracy: 0.6452

Epoch 00084: val_loss did not improve from 0.57795
Epoch 85/100
18/18 [==============================] - 1s 51ms/step - loss: 0.0589 - accuracy: 0.9800 - val_loss: 1.8095 - val_accuracy: 0.6452

Epoch 00085: val_loss di

NameError: name 'accuracy' is not defined

In [20]:
model_file_name =f'RCNN_model.h5'
sequence_model.save(model_file_name)

In [26]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)

Test video path: /home/isfrc/Desktop/cap/model-4/test1/handshaking/f_004_Trim.mp4
  handshaking: 99.80%
  typing:  0.10%
  sleeping:  0.10%
  closing_opening_door:  0.00%
  walking:  0.00%
